In [20]:
import numpy as np
a = np.array([[1,3],[2,4]])
print(table.max(axis=1))

NameError: name 'table' is not defined

**Problem:**

You are given the following dataset:
1. **Audible Data** : https://1drv.ms/u/s!AiqdXCxPTydhoog8ckLN-6Cw55fzIg?e=EWgZ5d

Your task is to:
- Find the problems with the datasets.
- Define the Data Quality Dimensions.
- Try to clean the datasets.

# Issues With the Data

### Dirty Data
Table - `in`
- name = column has [Book1, Book2, #1, #12] inserted in it `validity`
- name = language problem so letters like Ã, © in it `validity`
- name = has very dirty data that is not comprehensible like __Ð›ÐµÑÐ½Ñ‹Ðµ Ð¸ÑÑ‚Ð¾Ñ€Ð¸Ð¸__  maybe it should be in latin1 `validity`
- author = remove 'Writtenby:' also has same problem as above in name `validity`
- narrator = remove 'Nrratedby:' also has same problem as above in name `validity`
- language = not capitalize `consistency`
- time = have string value like 'Less than 1 minute' `validity`
- time = have hr and hrs; mins and min `consistency`

### Messy Data
Table - `in`

- time = not in right format
- releasedate = not in right format
- stars = should be splited in right format
- price = not in right format

In [298]:
import pandas as pd
import numpy as np
import re

In [338]:
audible = pd.read_excel('audible_uncleaned.xlsx',sheet_name='in')
audible.head()

,name,author,narrator,time,releasedate,language,stars,price
0,Geronimo Stilton #11 & #12,Writtenby:GeronimoStilton,Narratedby:BillLobely,2 hrs and 20 mins,2008-04-08 00:00:00,English,5 out of 5 stars34 ratings,468
1,The Burning Maze,Writtenby:RickRiordan,Narratedby:RobbieDaymond,13 hrs and 8 mins,2018-01-05 00:00:00,English,4.5 out of 5 stars41 ratings,820
2,The Deep End,Writtenby:JeffKinney,Narratedby:DanRussell,2 hrs and 3 mins,2020-06-11 00:00:00,English,4.5 out of 5 stars38 ratings,410
3,Daughter of the Deep,Writtenby:RickRiordan,Narratedby:SoneelaNankani,11 hrs and 16 mins,2021-05-10 00:00:00,English,4.5 out of 5 stars12 ratings,615
4,"The Lightning Thief: Percy Jackson, Book 1",Writtenby:RickRiordan,Narratedby:JesseBernstein,10 hrs,2010-01-13 00:00:00,English,4.5 out of 5 stars181 ratings,820


In [426]:
df = audible.copy()

In [427]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87489 entries, 0 to 87488
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         87489 non-null  object
 1   author       87489 non-null  object
 2   narrator     87489 non-null  object
 3   time         87489 non-null  object
 4   releasedate  87489 non-null  object
 5   language     87489 non-null  object
 6   stars        87489 non-null  object
 7   price        87489 non-null  object
dtypes: object(8)
memory usage: 5.3+ MB


In [428]:
df.shape

(87489, 8)

In [429]:
df['releasedate'] = pd.to_datetime(df['releasedate'])

In [430]:
def extract_nums(string):
    x = string.split()
    num = []
    for i in x:
        if i.isdigit():
            num.append(i)
    return num

In [431]:
df['time'] = df['time'].str.replace('min','minute',).str.replace('hrs','hours').str.replace('hr','hours').str.replace('Less than 1 minute ','1 minute')

In [432]:
df['hours'] = df[df['time'].str.contains('hours')]['time'].str.split().str.get(0)
df['minutes'] = df[df['time'].str.contains('minute')]['time'].str.split().str.get(-2)

In [433]:
df['minutes'].fillna('0',inplace=True)
df['hours'].fillna('0',inplace=True)

In [434]:
df['hours'] = df['hours'].str.replace('Less','0').astype('int')
df['minutes'] = df['minutes'].astype('int')

In [435]:
df['author'] = df['author'].str.replace('Writtenby:','')
df['narrator'] = df['narrator'].str.replace('Narratedby:','')

In [436]:
df['language'] = df['language'].str.capitalize()

In [437]:
df['stars'] = df['stars'].str.replace('stars',' ')

In [438]:
df['avg_ratings'] = df['stars'].str.split().str.get(0)
df['avg_ratings'] = df['avg_ratings'].str.replace('Not','0')
df['avg_ratings'] = df['avg_ratings'].astype(float)

In [439]:
df['total_ratings'] = df['stars'].str.split().str.get(-2)
df['total_ratings'] = df['total_ratings'].str.replace('rated','0')
df['total_ratings'] = df['total_ratings'].str.replace(',','').astype(int)

In [440]:
df['price'] = df['price'].replace('Free','0').astype(int)

In [443]:
df.drop(columns=['time','stars'],inplace=True)

In [444]:
df

,name,author,narrator,releasedate,language,price,hours,minutes,avg_ratings,total_ratings
0,Geronimo Stilton #11 & #12,GeronimoStilton,BillLobely,2008-04-08,English,468,2,20,5.0,34
1,The Burning Maze,RickRiordan,RobbieDaymond,2018-01-05,English,820,13,8,4.5,41
2,The Deep End,JeffKinney,DanRussell,2020-06-11,English,410,2,3,4.5,38
3,Daughter of the Deep,RickRiordan,SoneelaNankani,2021-05-10,English,615,11,16,4.5,12
4,"The Lightning Thief: Percy Jackson, Book 1",RickRiordan,JesseBernstein,2010-01-13,English,820,10,0,4.5,181
...,...,...,...,...,...,...,...,...,...,...
87484,Last Days of the Bus Club,ChrisStewart,ChrisStewart,2017-09-03,English,596,7,34,0.0,0
87485,The Alps,StephenO'Shea,RobertFass,2017-02-21,English,820,10,7,0.0,0
87486,The Innocents Abroad,MarkTwain,FloGibson,2016-12-30,English,938,19,4,0.0,0
87487,A Sentimental Journey,LaurenceSterne,AntonLesser,2011-02-23,English,680,4,8,0.0,0
